In [2]:
import json
import pandas as pd
import os

data_raw = "tpb_tweets_filtered_20220328.json"
data_in = 'tpb_tweets_simple-filter_labelled-prodigy-format_20220408.json'
data_out_n = 'tpb_tweets_labelled.xlsx'

#label_cols = ["Physical stuckness and Covid","Pandemic precarity","Blocked and derailed mobilities because of Covid","New mobilities in relation to Covid","Other"]
label_cols = ['blocked mobilities', 'new mobilities', 'pandemic precarity','physical stuckness', 'surrounding precarity and vulnerabilities']

data_d = os.path.join('..', 'data')

data_raw_p = os.path.join(data_d, data_raw)
data_in_p = os.path.join(data_d, data_in)
data_out_p = os.path.join(data_d, data_out_n)

In [3]:
with open(data_raw_p, 'r') as f:
    data = json.load(f)

In [4]:
df_raw = pd.DataFrame.from_records(data).drop_duplicates('id')

In [5]:
with open(data_in_p, 'r') as f:
    data = json.load(f)

In [6]:
df_in = pd.DataFrame.from_records(data).drop_duplicates(subset = 'id')

In [7]:
df_in = df_in.explode('accept')
df_in['accepted'] = 1
df_in_labeldummies = pd.merge(df_raw, df_in.pivot(index = 'id', columns="accept", values="accepted").reset_index().loc[:, ['id'] + label_cols].fillna(0), how = 'inner', on = 'id')

In [8]:
columns_neworder = ['created_at', 'name', 'username', 'text', 'tweet_link', 'covid'] + label_cols + ['id', 'hashtags', 'urls', 'retweet_count', 'reply_count',
                                                                                                     'like_count', 'quote_count', 'referenced_type', 'referenced_id']

In [9]:
df_out = df_in_labeldummies.loc[:, columns_neworder]

In [10]:
df_out.to_excel(data_out_p, index = False)